<a href="https://colab.research.google.com/github/kanchangiri67/OncoCist/blob/main/Example_object_localization_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all necessary libraries

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from joblib import dump, load
from google.colab import drive
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras import regularizers

# Step 1: Load the Dataset

In [ ]:
## Mount the drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Specify the directory
%cd '/content/drive/MyDrive/ColabNotebooks/MachineLearningFinalProject'

/content/drive/MyDrive/ColabNotebooks/MachineLearningFinalProject


### Load the Image Scalar

In [ ]:
image_scaler_reload = load('image_scaler.joblib')
image_scaler_reload

<function __main__.image_scaler(images_array)>

### Load the CNN Model

In [ ]:
cnn_model_reload = load('cnn_model.joblib')
cnn_model_reload

<Sequential name=sequential_1, built=True>

Deploy the machine learning models on Gradio

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import cv2

# Model Options
models = {
    "Convolutional Neural Network": cnn_model_reload
}

def transform_input(input_image, model_selection):
  input_image = cv2.resize(input_image, (128, 128))
  images_array = np.array([input_image])

  if model_selection == "Convolutional Neural Network":
    print(input_image.shape)
    image_transform = image_scaler_reload(images_array)
    image_transform = image_transform.reshape(1, 128, 128, 3)
  else:
    image_transform = image_scaler_reload(images_array)
    image_transform = pca_reload.transform(image_transform)
  return image_transform

def apply_model(input_image, model_selection):
  image_transform = transform_input(input_image, model_selection)
  predicted_values = models[model_selection].predict(image_transform)

  plt.imshow(input_image)

  box_x = predicted_values[0][0]
  box_y = predicted_values[0][1]
  box_width = predicted_values[0][2]
  box_height = predicted_values[0][3]

  plt.gca().add_patch(matplotlib.patches.Rectangle((box_x, box_y), box_width, box_height, ec='b', fc='none'))
  plt.axis('off')

  buf = BytesIO()
  plt.savefig(buf, format='png')
  buf.seek(0)
  plt.close()

  output_image = plt.imread(buf)
  buf.close()

  return output_image

## Input 1: Get Image
input_image = gr.Image()

## Input 2: Model Selection
model_name = gr.Dropdown(choices=list(models.keys()), label="Select ML Model for Object Localization")

## Output Image
output_image = gr.Image()

## Create the interface
interface = gr.Interface(
              fn=apply_model,
              inputs=[input_image, model_name],
              outputs=output_image,
              title = "Object Localization",
              description= "Submit an image and select a machine learning model to see a bounding box around the stop sign. "
              )

## Launch the interface
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9eed781f82719d57e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


(128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 749ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9eed781f82719d57e7.gradio.live
